<a href="https://colab.research.google.com/github/4dsolutions/clarusway_data_analysis/blob/main/python_warm_up/object_oriented2.ipynb"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open and Execute in Google Colaboratory"></a><br/>
[![nbviewer](https://raw.githubusercontent.com/jupyter/design/master/logos/Badges/nbviewer_badge.svg)](https://nbviewer.org/github/4dsolutions/clarusway_data_analysis/blob/main/python_warm_up/warmup_object_oriented2.ipynb)


# Class and Static Methods, Properties

<a data-flickr-embed="true" href="https://www.flickr.com/photos/kirbyurner/52563704012/in/album-72177720296706479/" title="LMS Dashboard"><img src="https://live.staticflickr.com/65535/52563704012_71ef4beb8a_b.jpg" width="1024" height="354" alt="LMS Dashboard"></a><script async src="//embedr.flickr.com/assets/client-code.js" charset="utf-8"></script>

Python Warm-up Notebooks:

*  [Introduction to Python](warmup_python_intro.ipynb)
*  [3rd Party Libraries](warmup_3rd_party_datascience.ipynb)
*  [Object Types](warmup_data_structures.ipynb)
*  [Object Oriented Paradigm](warmup_object_oriented.ipynb)
*  [Calling Callables and Type Checking](warmup_callables.ipynb)
*  [Class and Static Methods, Properties](warmup_object_oriented2.ipynb)  (you are here)
*  [SQLite3 and Context Managers](warmup_object_sql.ipynb)
*  [Iterators and Generators](warmup_generators.ipynb) 

## What is self?

What may seem mysterious at first, when inspecting class code, is the meaning of `self`, which is not a keyword so much as a placeholder.  Other names could go here, but the convention is to use `self` without exception.

Let's use `me` instead.

In [1]:
class Animal:

    version = 1.2
    
    def __init__(me):       # a me is born
        me.stomach = []   # ... with an empty stomach
        
    def eat(me, food):    # a verb, something I, an Animal self, do
        me.stomach.append(food)    

class Cat(Animal):
    
    def __call__(self, n):
        return "Meow! " * n

    def __repr__(self):     
        return "Cat at " + str(id(self)) 
    
class Donkey(Animal):
    
    def __call__(self, n):
        return "HeeHaw! " * n

    def __repr__(self):     
        return "Donkey at " + str(id(self)) 

In [2]:
kitty = Cat()   # __init__ is in the parent Animal class
kitty.eat("mouse")
kitty(5)

'Meow! Meow! Meow! Meow! Meow! '

In [3]:
kitty.stomach

['mouse']

In [4]:
kong = Donkey()
kong.eat("hay")  # eat is in the parent class
kong(5)

'HeeHaw! HeeHaw! HeeHaw! HeeHaw! HeeHaw! '

What's going on with the three classes?  The Cat and Donkey classes are *subclasses* of the Animal class.  We may say that Animal is the parent class, or super class, of Cat and Donkey.

What this means is that if a method such as `eat` cannot be found inside `Cat`, look higher in the class hierarchy.  What is `Cat` descended from?  `Animal`.  So look there.  Keep ascending the class hierarchy (picture a tree) all the way up to the top, which is `object` (all ordinary classes inherit from object).

In [5]:
Cat.__mro__   # when looking for a method or attribute, follow this sequence

(__main__.Cat, __main__.Animal, object)

`__mro__` stands for "method resolution order" and shows in what order the class hierarchy will be consulted when looking for a matching method or attribute name.

In [6]:
issubclass(Cat, Animal)

True

In [7]:
isinstance(kitty, Cat)

True

In [8]:
isinstance(kong, Cat)

False

In [9]:
isinstance(kong, Animal)

True

In [10]:
issubclass(Donkey, Animal)

True

When a new cat or donkey is needed, we call Cat or Donkey, but neither has an explicit initializer.  Therefore Python searches in their common parent class, Animal, and here is where a self is born, or call it a me.  The `__init__` method has no `return` because at always gives back the same thing:  a new self, a new instance.

Inside a given class, the particular instance we are deal with is known as `self`.  All the selves, all the instances, share the same class code in memory.  When a specific cat eats, or a specific donkey, the shared `eat` method is always used.  But Python knows which instance or self is eating, and keeps that object handy at all times.

We can see the self in action if we omit it, and call a class directly.  If we do that, we have to supply the self explicitly.

In [11]:
Animal.eat(kong, 'gum drop')  # calling Animal.eat directly, so pass the self

In [12]:
kong.stomach

['hay', 'gum drop']

In [13]:
kitty.eat('sushi')  # the self is already known, so we don't pass it

In [14]:
kitty.stomach

['mouse', 'sushi']

## @staticmethod and @classmethod

Suppose we do not want or need to work with a specific self or instance.  We would like to change the attributes of an entire class, irrespective of its instances.  That's where `classmethod` comes in handy.  

By using the `@classmethod` decorator above a method, the method becomes "about the class" and its leftmost argument will now be the class itself instead of the instance (self).

A `staticmethod` does not even care about what class it's in.

In [15]:
import random

class Dog(Animal):
    
    @staticmethod
    def version():
        return "2.1"
    
    tricks = ['play dead', 'roll over', 'fetch', 'sit']  # class level
    
    @classmethod
    def add_trick(cls, trick):
        cls.tricks.append(trick)
#   add_trick = classmethod(add_trick) <-- same as using a decorator
        
    def do_trick(self):
        return random.choice(self.tricks)  # finds tricks at the Dog level
    
    def __call__(self, n):
        return "Bark! " * n

    def __repr__(self):     
        return "Donkey at " + str(id(self)) 

In [16]:
fido = Dog()
rover = Dog()
fido.do_trick()

'sit'

In [17]:
fido.__dict__  # no knowledge of tricks at the instance level

{'stomach': []}

In [18]:
fido.add_trick('lie down')
Dog.tricks

['play dead', 'roll over', 'fetch', 'sit', 'lie down']

In [19]:
rover.tricks  # now rover knows them too

['play dead', 'roll over', 'fetch', 'sit', 'lie down']

In [20]:
rover.version()

'2.1'

## @property

The @property decorator allows us to disguise a method call as if merely setting an attribute.

Lets define a Circle type that keeps track of its own radius, circumference, and area.  Changing any one of these attributes will change the two others.  

Let's just operate with such a Circle without first looking at source code.

In [21]:
from code_demos_circle import Circle
import math

In [22]:
the_circle = Circle(5)                  # initialize
print("the_circle:", the_circle)
print("Area: ", the_circle.area)
the_circle.area = 50                    # looks like simple assignment, triggers the setter
print("Radius when Area=50:", the_circle.radius)
the_circle.circumference = math.pi * 2  # triggers setter
print("Radius with circumference is 2*pi: {}".format(the_circle.radius))

the_circle: Circle(radius = 5)
Area:  78.53981633974483
Radius when Area=50: 3.989422804014327
Radius with circumference is 2*pi: 1.0
